In [1]:
from Resources.lib import *
from Resources import lib as lib
from Resources import model_package as MODEL

2024-02-11 15:03:48,902 numexpr.utils INFO Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-02-11 15:03:48,903 numexpr.utils INFO NumExpr defaulting to 8 threads.
2024-02-11 15:03:51.436227: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 15:03:51.436450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 15:03:51.480396: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-11 15:03:51.576754: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instruction

In [2]:
###----------------
### Some parametersimport warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd
import tensorflow as tf
###----------------

inpDir = './upload'
outDir = '../DATA'
subDir = 'csvs'
audDir = "AUDIO"

RANDOM_STATE = 24               # REMEMBER: to remove at the time of promotion to production
np.random.seed(RANDOM_STATE)    # Set Random Seed for reproducible  results

EPOCHS = 11                     # number of epochs
ALPHA = 0.001                   # learning rate
NUM_SAMPLES = 1280              # How many samples we want to generate 
NOISE = 0.2                     # Noise to be introduced in the data
TEST_SIZE = 0.2
BATCH_SIZE = 26
PATIENCE = 20

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 8),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

CMAP = 'coolwarm' # plt.cm.Spectral

plt.rcParams.update(params)

In [3]:
df = pd.read_csv("./DATA/csvs/trump-original_REAL.csv")
df = df.drop(columns="LABEL")
df

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.358431,0.047544,3542.994033,2472.263393,6142.101496,0.229714,-183.50685,34.215317,-25.610510,5.232159,...,-8.369213,-14.175573,-15.062232,-2.411484,-2.136527,4.616875,-3.939896,-5.642972,-8.739244,-9.369484
1,0.398888,0.013980,2956.441093,2525.691815,5812.986062,0.171032,-350.51126,58.881863,-23.915743,13.723481,...,-9.138926,-9.554494,-8.560787,-2.228990,-14.121047,-0.228751,-2.585347,-3.479855,-10.985689,-4.435715
2,0.277066,0.028064,2216.672834,2266.495699,4373.197798,0.103405,-328.82242,84.158720,-20.430117,6.658457,...,-4.212953,-9.122508,-6.528575,5.203862,-6.371897,-2.242783,-7.658451,-6.257457,-6.273143,-3.551254
3,0.359222,0.035303,2968.641140,2467.412969,5483.625932,0.178067,-265.03745,61.225735,-20.567955,-0.498796,...,-6.021339,-6.477277,-14.361066,0.192268,-5.031093,-0.998947,-3.826711,-2.932528,-9.419311,-3.316403
4,0.329339,0.024873,2111.408670,2129.420379,3848.325972,0.105125,-283.91092,91.656950,-41.411020,-11.166985,...,-9.240157,-2.310187,-10.009682,-3.353667,-7.186926,-0.003199,-9.301209,-5.654606,-5.261727,2.498810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.380453,0.032703,2666.514180,2112.847586,4668.300559,0.163064,-289.70065,79.301820,-34.634937,-2.610724,...,0.092124,1.182775,-15.351231,-2.042419,-9.185846,-2.628170,-11.762791,-5.393684,-8.502969,-2.474540
597,0.317955,0.036571,2310.894483,2100.396248,4078.339733,0.125388,-264.99075,87.606865,-33.935280,10.865050,...,4.615997,-5.449326,-13.275034,5.052628,-9.755420,-6.369123,-11.137697,-3.693965,-7.176722,-5.510276
598,0.417653,0.041490,1943.153338,1824.786318,3402.001398,0.105247,-205.10083,105.617240,-65.244415,-11.780135,...,-6.529221,-1.176161,-11.554144,-4.132590,-11.246901,0.278846,-11.095168,-3.507075,-4.943163,3.291278
599,0.401598,0.037857,2042.405086,1803.789276,3407.874090,0.118852,-200.52167,85.288734,-90.497860,-33.224705,...,1.590815,4.300089,-3.402193,-4.930651,-13.453516,-5.114317,-7.686624,1.444214,-2.637488,2.451039


In [7]:
df = lib.create_DataFrame("DATA/KAGGLE/AUDIO/FAKE/linus-to-trump.wav" , segment_length=1)

model = MODEL.Model()
model.buid_model()
model.load_weights("Weights/demo_weigths")
result = model.predict(df)

2024-02-09 17:42:42,892 LIB INFO Feature Extraction STARTED
2024-02-09 17:42:58,149 LIB INFO Feature Extraction SUCCESSFUL
2024-02-09 17:42:58,158 LIB INFO DataFrame creation SUCCESSFUL
2024-02-09 17:42:58,170 tensorflow WARNING Layer gru_2 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


In [8]:
result

' 98.25 Fake%'

In [6]:
lib.TF_Predict(df)

2024-02-09 17:42:14,500 tensorflow WARNING Layer gru_1 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


' 89.68 Fake%'

In [5]:
from pydub import AudioSegment

def convert_mp3_to_wav(mp3_file, wav_file):
    try:
        # Load the MP3 file
        audio = AudioSegment.from_mp3(mp3_file)
        
        # Export the audio to WAV
        audio.export(wav_file, format="wav")
        
        print(f"MP3 file '{mp3_file}' converted to WAV file '{wav_file}' successfully.")
    except Exception as e:
        print(f"Error converting MP3 to WAV: {e}")


In [9]:
# Example usage:
convert_mp3_to_wav("DATA/DEMONSTRATION/DEMONSTRATION/linus-original-DEMO.mp3", "Upload/linus-original-DEMO.wav")

MP3 file 'DATA/DEMONSTRATION/DEMONSTRATION/linus-original-DEMO.mp3' converted to WAV file 'Upload/linus-original-DEMO.wav' successfully.


In [2]:
# !pip install pydub

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
file = "linus-original-DEMO.mp3"

In [12]:
"Upload/"+file[:-4] + ".wav"

'Upload/linus-original-DEMO.wav'

In [7]:
import os

def delete_file(file_path):
    """
    Deletes a file given its file path.

    Parameters:
    - file_path (str): The path to the file to be deleted.

    Returns:
    - bool: True if the file was successfully deleted, False otherwise.
    """
    try:
        os.remove(file_path)
        print(f"File '{file_path}' deleted successfully.")
        return True
    except OSError as e:
        print(f"Error: {e.filename} - {e.strerror}.")
        return False

# Example usage:
file_path = "example.txt"  # Replace this with the path of the file you want to delete

File 'test.wav' deleted successfully.


True

In [42]:
Orig_path = "DATA/new/Real/common_voice_en_38487408.mp3"
out_path = "test.wav"
def Convert_mp3_to_csv(in_path , csv_path):
    
    lib.convert_mp3_to_wav(Orig_path , out_path)
    df = create_DataFrame(out_path , segment_length=1)
    df['LABEL'] = ["REAL" for _ in range(len(df))]
    df.to_csv(csv_path, index=False)
    delete_file(out_path)

In [41]:
Convert_mp3_to_csv(Orig_path , "DATA/new/common_voice_en_38487408.csv")

2024-02-11 16:44:35,556 LIB INFO Feature Extraction STARTED


MP3 file 'DATA/new/Real/common_voice_en_38487408.mp3' converted to WAV file 'test.wav' successfully.


2024-02-11 16:44:35,781 LIB INFO Feature Extraction SUCCESSFUL
2024-02-11 16:44:35,783 LIB INFO DataFrame creation SUCCESSFUL


File 'test.wav' deleted successfully.


,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,LABEL
0,0.437773,0.000082,2975.323539,1897.680217,5098.230535,0.122991,-1013.804932,26.898451,-1.980751,5.975697,...,-1.234398,0.462547,-3.723354,-1.217206,-3.813755,0.810404,-1.823659,-0.836677,-0.461611,REAL
1,0.406069,0.011585,1194.170163,1184.333580,2033.419522,0.068459,-474.920990,176.539536,-41.408447,0.977752,...,-10.689574,-5.614941,-7.513553,-1.901567,-3.176272,-8.451154,-6.318482,-1.820427,-12.978436,REAL
2,0.376788,0.015634,1156.892201,1178.945769,2072.081410,0.076926,-497.315918,139.916458,-9.572052,47.944969,...,-21.657745,-5.270089,-0.882757,-1.587419,-5.773855,-4.572124,-0.221385,3.072713,-5.822569,REAL
3,0.393805,0.011719,1819.236491,1692.715720,3341.561612,0.166726,-527.338135,109.996086,4.836823,29.546980,...,-12.671713,-2.796919,-4.541454,-3.408234,-7.628582,-1.604970,-3.321325,-0.436963,-3.380804,REAL
4,0.516238,0.000242,3090.135696,2621.831108,6119.834206,0.584850,-855.735901,71.932808,-12.793786,1.678472,...,-4.941845,-6.681929,-2.445552,-2.829204,-0.937784,-2.665725,0.897055,0.290788,4.563674,REAL
5,0.428407,0.002185,2081.935501,2045.651697,4311.740594,0.263209,-676.636963,121.627907,-38.316471,27.902523,...,-9.823982,-1.159283,-4.709579,-6.459794,-0.357279,-8.664439,-13.258037,-3.448294,-3.956877,REAL


In [ ]:
import os

def delete_file(file_path):
    """
    Deletes a file given its file path.

    Parameters:
    - file_path (str): The path to the file to be deleted.

    Returns:
    - bool: True if the file was successfully deleted, False otherwise.
    """
    try:
        os.remove(file_path)
        print(f"File '{file_path}' deleted successfully.")
        return True
    except OSError as e:
        print(f"Error: {e.filename} - {e.strerror}.")

def Convert_mp3_to_csv(in_path , csv_path):
    
    lib.convert_mp3_to_wav(Orig_path , out_path)
    df = create_DataFrame(out_path , segment_length=1)
    df['LABEL'] = ["REAL" for _ in range(len(df))]
    df.to_csv(csv_path, index=False)
    delete_file(out_path)

def mp3dir_to_csvdir(in_dir , csv_dir):
    paths = os.listdir(in_dir)
    for i in range(501):
        filepath = in_dir+paths[i]
        csvpath = csv_dir+paths[i][:-4]+"_REAL"+".csv"
        Convert_mp3_to_csv(in_path=filepath , csv_path=csvpath)
        
mp3dir_to_csvdir(in_dir="DATA/new/Real",csv_dir="DATA/new/")

: 

In [ ]:
# def Convert_mp3_to_csv(in_path , csv_path):
    
#     lib.convert_mp3_to_wav(Orig_path , out_path)
#     df = create_DataFrame(out_path , segment_length=1)
#     df['LABEL'] = ["REAL" for _ in range(len(df))]
#     df.to_csv(csv_path, index=False)
#     delete_file(out_path)

# def mp3dir_to_csvdir(in_dir , csv_dir):
#     paths = os.listdir(in_dir)
#     for i in paths:
#         filepath = in_dir+i
#         csvpath = csv_dir+i[:-4]+"_REAL"+"csv"
#         Convert_mp3_to_csv(in_path=filepath , csv_path=csvpath)
        
# mp3dir_to_csvdir(in_dir="DATA/new/Real",csv_dir="DATA/new/")